In [30]:
import torch
import deepsensor
import deepsensor.torch
from deepsensor.train import set_gpu_default_device
set_gpu_default_device()

In [31]:
from deepsensor.train import Trainer
from deepsensor.model import ConvNP
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from deepsensor.data import DataProcessor, TaskLoader
from tqdm import tqdm
from scipy.special import sph_harm

# Load first three files from dataset
# Set the path to your Zarr store on GCS
data_path = '/nfs/turbo/seas-dannes/urop-2024-bias/cfs-forecasts/cfs_2012-2023.zarr'

# Open the Zarr store using Xarray
ds0 = xr.open_zarr(data_path, consolidated=True)

# Check the contents of the dataset
ds0
#ds = ds.sel(time=ds.time.dt.hour == 0)

#ds = ds.sel(time=slice("2012-01-01", "2015-12-31"))  # Select only the first three files




<xarray.Dataset> Size: 132GB
Dimensions:            (time: 16847, lead: 10, latitude: 181, longitude: 360)
Coordinates:
  * latitude           (latitude) float64 1kB -90.0 -89.0 -88.0 ... 89.0 90.0
  * lead               (lead) int64 80B 0 1 2 3 4 5 6 7 8 9
  * longitude          (longitude) float64 3kB 0.0 1.0 2.0 ... 357.0 358.0 359.0
  * time               (time) datetime64[ns] 135kB 2012-01-01 ... 2024-01-01T...
Data variables:
    LHTFL_surface      (time, lead, latitude, longitude) float32 44GB dask.array<chunksize=(1, 5, 91, 180), meta=np.ndarray>
    SHTFL_surface      (time, lead, latitude, longitude) float32 44GB dask.array<chunksize=(1, 5, 91, 180), meta=np.ndarray>
    TMP_2maboveground  (time, lead, latitude, longitude) float32 44GB dask.array<chunksize=(1, 5, 91, 180), meta=np.ndarray>

In [32]:
ds = ds0.isel(lead=0, drop=True)
ds = ds.sortby("time")
ds

<xarray.Dataset> Size: 13GB
Dimensions:            (time: 16847, latitude: 181, longitude: 360)
Coordinates:
  * latitude           (latitude) float64 1kB -90.0 -89.0 -88.0 ... 89.0 90.0
  * longitude          (longitude) float64 3kB 0.0 1.0 2.0 ... 357.0 358.0 359.0
  * time               (time) datetime64[ns] 135kB 2012-01-01 ... 2024-01-01T...
Data variables:
    LHTFL_surface      (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    SHTFL_surface      (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    TMP_2maboveground  (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>

In [33]:
# Apply bias correction function

def random_noise_bias(data, scale=3):
    return data + np.random.uniform(-scale, scale, data.shape)


In [34]:
print(ds.sel(time=slice("2012-01-01T00:00:00.000000000", "2012-01-03T00:00:00.000000000")))

<xarray.Dataset> Size: 7MB
Dimensions:            (time: 9, latitude: 181, longitude: 360)
Coordinates:
  * latitude           (latitude) float64 1kB -90.0 -89.0 -88.0 ... 89.0 90.0
  * longitude          (longitude) float64 3kB 0.0 1.0 2.0 ... 357.0 358.0 359.0
  * time               (time) datetime64[ns] 72B 2012-01-01 ... 2012-01-03
Data variables:
    LHTFL_surface      (time, latitude, longitude) float32 2MB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    SHTFL_surface      (time, latitude, longitude) float32 2MB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    TMP_2maboveground  (time, latitude, longitude) float32 2MB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>


In [35]:
data_processor = DataProcessor(x1_name="latitude", x2_name="longitude")
_ = data_processor(ds.sel(time=slice("2012-01-01T00:00:00.000000000", "2012-01-31T00:00:00.000000000")))
ds_processed = data_processor(ds)
print(data_processor)

DataProcessor with normalisation params:
{'LHTFL_surface': {'method': 'mean_std',
                   'params': {'mean': 66.60301208496094,
                              'std': 65.60218048095703}},
 'SHTFL_surface': {'method': 'mean_std',
                   'params': {'mean': 6.118175506591797,
                              'std': 31.839258193969727}},
 'TMP_2maboveground': {'method': 'mean_std',
                       'params': {'mean': 276.7368469238281,
                                  'std': 20.317398071289062}},
 'coords': {'time': {'name': 'time'},
            'x1': {'map': (-90.0, 269.0), 'name': 'latitude'},
            'x2': {'map': (0.0, 359.0), 'name': 'longitude'}}}


In [36]:
print(ds_processed)

<xarray.Dataset> Size: 13GB
Dimensions:            (time: 16847, x1: 181, x2: 360)
Coordinates:
  * time               (time) datetime64[ns] 135kB 2012-01-01 ... 2024-01-01T...
  * x1                 (x1) float64 1kB 0.0 0.002786 0.005571 ... 0.4986 0.5014
  * x2                 (x2) float64 3kB 0.0 0.002786 0.005571 ... 0.9972 1.0
Data variables:
    LHTFL_surface      (time, x1, x2) float32 4GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    SHTFL_surface      (time, x1, x2) float32 4GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>
    TMP_2maboveground  (time, x1, x2) float32 4GB dask.array<chunksize=(1, 91, 180), meta=np.ndarray>


In [37]:
train_range = ("2012-01-01", "2014-12-31")
val_range = ("2015-01-01", "2015-12-31")

In [38]:
def add_bias_function(data, bias_function, **kwargs):
    return data + bias_function(data, **kwargs)

# Define TaskLoader
biased_contexts = [
    #add_bias_function(ds_processed["TMP_2maboveground"], spherical_harmonic_bias),
    add_bias_function(ds_processed["TMP_2maboveground"], random_noise_bias),
    #add_bias_function(ds_processed["TMP_2maboveground"], linear_trend_bias),
    #add_bias_function(ds_processed["TMP_2maboveground"], periodic_step_bias),
]

task_loader = TaskLoader(
    context=biased_contexts,  
    target=ds_processed["TMP_2maboveground"],
)

print(task_loader)

TaskLoader(1 context sets, 1 target sets)
Context variable IDs: (('TMP_2maboveground',),)
Target variable IDs: (('TMP_2maboveground',),)


In [39]:
import time
import sys
def gen_tasks(dates, progress=True):
    tasks = []
    for date in tqdm(dates):
        date = np.datetime64(date)  # Ensure consistent datetime format
        try:
            task = task_loader(date, context_sampling="all", target_sampling="all")  
            tasks.append(task)
        except KeyError:
            print(f"Skipping date {date} as it is not found in dataset.")
        sys.stdout.flush()
    print(f"Finished generating {len(tasks)} tasks.")
    return tasks


# Define the ConvNP model
model = ConvNP(data_processor, task_loader, dim_yc=(1,))

# Train the model
trainer = Trainer(model)

dim_yt inferred from TaskLoader: 1
dim_aux_t inferred from TaskLoader: 0
internal_density inferred from TaskLoader: 359
encoder_scales inferred from TaskLoader: [np.float32(0.0013927576)]
decoder_scale inferred from TaskLoader: 0.002785515320334262


In [40]:
ds["time"] = pd.to_datetime(ds["time"].values).strftime("%Y-%m-%d %H:%M:%S")

In [41]:
print(ds.time)

<xarray.DataArray 'time' (time: 16847)> Size: 135kB
array(['2012-01-01 00:00:00', '2012-01-01 06:00:00', '2012-01-01 12:00:00',
       ..., '2024-01-01 06:00:00', '2024-01-01 12:00:00',
       '2024-01-01 18:00:00'], dtype=object)
Coordinates:
  * time     (time) object 135kB '2012-01-01 00:00:00' ... '2024-01-01 18:00:00'


In [42]:
losses = []
val_rmses = []
train_rmses = []

In [43]:
def compute_val_rmse(model, val_tasks):
    errors = []
    target_var_ID = task_loader.target_var_IDs[0][0]  # assume 1st target set and 1D
    for task in val_tasks:
        mean = data_processor.map_array(model.mean(task), target_var_ID, unnorm=True)
        true = data_processor.map_array(task["Y_t"][0], target_var_ID, unnorm=True)
        errors.extend(np.abs(mean - true))
    return np.sqrt(np.mean(np.concatenate(errors) ** 2))


In [44]:
def compute_train_rmse(model, train_tasks):
    errors = []
    context_var_ID = task_loader.context_var_IDs[0][0]  # assume 1st target set and 1D
    for task in train_tasks:
        mean = data_processor.map_array(model.mean(task), context_var_ID, unnorm=True)
        true = data_processor.map_array(task["Y_t"][0], context_var_ID, unnorm=True)
        errors.extend(np.abs(mean - true))
    return np.sqrt(np.mean(np.concatenate(errors) ** 2))


In [45]:
val_dates = pd.date_range(val_range[0], val_range[1])
val_tasks = gen_tasks(val_dates)
_ = model(val_tasks[0])
print(f"Model has {deepsensor.backend.nps.num_params(model.model):,} parameters")

100%|██████████| 365/365 [03:00<00:00,  2.02it/s]

Finished generating 365 tasks.
Model has 1,027,844 parameters


In [46]:
# Train model
#deepsensor_folder = "/home/whruiray/deepsensor_config/"


val_rmse_best = np.inf
train_rmse_best = np.inf


trainer = Trainer(model, lr=5e-5)
train_tasks = gen_tasks(pd.date_range(train_range[0], train_range[1]), progress=False)

  5%|▌         | 60/1096 [00:29<08:30,  2.03it/s]

Skipping date 2012-03-01T00:00:00.000000 as it is not found in dataset.


 16%|█▌        | 176/1096 [01:26<07:35,  2.02it/s]

Skipping date 2012-06-25T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-26T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-27T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-28T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-29T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-06-30T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-01T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-02T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-03T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-04T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-05T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-06T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-07T00:00:00.000000 as it is not found in dataset.
Skipping date 2012-07-08T00:00:00.000000 as it is not found in d

 20%|█▉        | 215/1096 [01:27<00:35, 24.72it/s]

...

Skipping date 2013-06-17T00:00:00.000000 as it is not found in dataset.
Skipping date 2013-06-18T00:00:00.000000 as it is not found in dataset.


 98%|█████████▊| 1069/1096 [08:28<00:13,  2.03it/s]

Skipping date 2014-12-05T00:00:00.000000 as it is not found in dataset.
Skipping date 2014-12-06T00:00:00.000000 as it is not found in dataset.


100%|██████████| 1096/1096 [08:40<00:00,  2.11it/s]

Finished generating 1049 tasks.


In [ ]:
for epoch in tqdm(range(10)):
    #train_tasks = gen_tasks(pd.date_range(train_range[0], train_range[1]), progress=False)
    
    print(f"Training batch for Epoch {epoch+1} started...")
    sys.stdout.flush()
    
    batch_losses = trainer(train_tasks)
    
    
    losses.append(np.mean(batch_losses))
    print(f"Training batch for Epoch {epoch+1} complete...")
    sys.stdout.flush()
   
    train_rmses.append(compute_train_rmse(model, train_tasks))
    print(f"Epoch {epoch+1} - Loss: {losses[-1]:.4f}, Validation RMSE: {train_rmses[-1]:.4f}")
    print(f"Epoch {epoch+1} completed in {time.time()} seconds.")
    sys.stdout.flush()
    if train_rmses[-1] < train_rmse_best:
        train_rmse_best = train_rmses[-1]

    val_rmses.append(compute_val_rmse(model, val_tasks))
    print(f"Epoch {epoch+1} - Loss: {losses[-1]:.4f}, Validation RMSE: {val_rmses[-1]:.4f}")
    print(f"Epoch {epoch+1} completed in {time.time()} seconds.")
    sys.stdout.flush()
    if val_rmses[-1] < val_rmse_best:
        val_rmse_best = val_rmses[-1]

        #model.save(deepsensor_folder)

  0%|          | 0/10 [00:00<?, ?it/s]

Training batch for Epoch 1 started...
Training batch for Epoch 1 complete...
Epoch 1 - Loss: -0.9045, Validation RMSE: 1.8494
Epoch 1 completed in 1744054046.0715837 seconds.
Epoch 1 - Loss: -0.9045, Validation RMSE: 1.8521
Epoch 1 completed in 1744054082.5479188 seconds.


 10%|█         | 1/10 [06:45<1:00:50, 405.63s/it]

Training batch for Epoch 2 started...
Training batch for Epoch 2 complete...
Epoch 2 - Loss: -1.1201, Validation RMSE: 1.7431
Epoch 2 completed in 1744054451.9598906 seconds.
Epoch 2 - Loss: -1.1201, Validation RMSE: 1.7435
Epoch 2 completed in 1744054488.4450562 seconds.


 20%|██        | 2/10 [13:31<54:06, 405.79s/it]  

...

Training batch for Epoch 6 started...
Training batch for Epoch 6 complete...
Epoch 6 - Loss: -1.3011, Validation RMSE: 1.6082
Epoch 6 completed in 1744056078.1311364 seconds.
Epoch 6 - Loss: -1.3011, Validation RMSE: 1.6120
Epoch 6 completed in 1744056114.6752253 seconds.


 60%|██████    | 6/10 [40:37<27:05, 406.48s/it]

Training batch for Epoch 7 started...
Training batch for Epoch 7 complete...
Epoch 7 - Loss: -1.3337, Validation RMSE: 1.5117
Epoch 7 completed in 1744056484.8232505 seconds.
Epoch 7 - Loss: -1.3337, Validation RMSE: 1.5204
Epoch 7 completed in 1744056521.362485 seconds.


 70%|███████   | 7/10 [47:24<20:19, 406.55s/it]

Training batch for Epoch 8 started...
Training batch for Epoch 8 complete...
Epoch 8 - Loss: -1.3677, Validation RMSE: 1.4851
Epoch 8 completed in 1744056891.689859 seconds.


In [ ]:
import os
import matplotlib.pyplot as plt

# Plot training metrics with adjusted scales
fig, ax1 = plt.subplots(figsize=(10, 6))

# Smooth the loss values (using simple moving average)
window_size = 5
smoothed_losses = np.convolve(losses, np.ones(window_size)/window_size, mode='valid')

# Plot training loss
ax1.plot(range(len(smoothed_losses)), smoothed_losses, 'b-', label='Training Loss')
ax1.set_xlabel('Epochs')
ax1.set_ylabel('Loss', color='b')
ax1.tick_params('y', colors='b')

# Create a second y-axis for validation RMSE
ax2 = ax1.twinx()
ax2.plot(range(len(val_rmses)), val_rmses, 'r-', label='Validation RMSE')
ax2.set_ylabel('RMSE', color='r')
ax2.tick_params('y', colors='r')

# Adjust scales to show downward trend
if len(losses) > 0:
    loss_min, loss_max = min(smoothed_losses), max(smoothed_losses)
    ax1.set_ylim(loss_min - 0.1*(loss_max-loss_min), loss_max + 0.1*(loss_max-loss_min))

if len(val_rmses) > 0:
    rmse_min, rmse_max = min(val_rmses), max(val_rmses)
    ax2.set_ylim(rmse_min - 0.1*(rmse_max-rmse_min), rmse_max + 0.1*(rmse_max-rmse_min))

plt.title('Training Loss and Validation RMSE')
fig.tight_layout()
plt.savefig("fig/constant_bias_metrics.png")
plt.close()


In [ ]:

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(losses)
axes[1].plot(val_rmses)
_ = axes[0].set_xlabel("Epoch")
_ = axes[1].set_xlabel("Epoch")
_ = axes[0].set_title("Training loss")
_ = axes[1].set_title("Validation RMSE")

plt.tight_layout()
plt.savefig("fig/constant_bias_loss_and_rmse.png")
plt.close()

In [ ]:
import deepsensor
from deepsensor import plot
import matplotlib.pyplot as plt

# Select a sample date for visualization
sample_date = "2012-01-01"
sample_task = task_loader(sample_date, context_sampling="all", target_sampling="all")

task_loader_random_noise = TaskLoader(
    context=biased_contexts,
    target=ds_processed["TMP_2maboveground"],
)
# Plot context (biased) and target (original) data
fig = deepsensor.plot.task(sample_task, task_loader=task_loader_random_noise)
fig.savefig("fig/random_noise_context_target_plot.png")
plt.close(fig)

In [ ]:
# Step 1: Choose the target date for prediction
target_date = "2016-01-15"  # Example date in the 2016-2020 range

# Step 2: Load the task for the target date (context and target data)
task = task_loader(target_date, context_sampling="all", target_sampling="all")

# Step 3: Run the model to get predictions
# Note that X_t is just passed as ds. It gets the xarray structure from ds. 
pred_val = model.predict(task, X_t=ds)  # Just pass the task object

# Step 4: Extract the relevant variable for predictions (e.g., 'APCP_surface' or 'TMP_2maboveground')
# Assuming 'TMP_2maboveground' is the variable you're predicting
predxr = pred_val['TMP_2maboveground']

# Step 5: Plot the mean prediction
predxr['mean'].plot(cmap='viridis')  # Plot mean prediction
plt.title(f"Prediction for {target_date}")
plt.savefig(f"fig/prediction_for_random_noise_{target_date}_mean.png")
plt.close()


# Step 6: Plot the standard deviation as well
predxr['std'].plot(cmap='viridis')  # Plot standard deviation
plt.title(f"Standard Deviation for {target_date}")
plt.savefig(f"fig/prediction_for_random_noise_{target_date}_std.png")
plt.close()

In [ ]:
pred_val